In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
data = pd.read_csv("worlds2020.csv", delimiter=";")
data_2 = pd.read_csv("worlds2020.csv", delimiter=";")
data = data.sample(frac=1).reset_index(drop=True)

In [3]:
data_target = data["Result"].values
data = data.drop(["Result"], axis=1)

In [4]:
data.head(50)

,Team,Wins,Losses,Top,Jungle,Mid,Ad carry,Support,Side
0,TSM,0,2,Shen,Graves,LeBlanc,Ezreal,Bard,Blue
1,Fnatic,3,1,Volibear,Hecarim,Orianna,Ashe,Leona,Blue
2,MCX,1,2,Jayce,Volibear,Syndra,Ashe,Nautilus,Blue
3,TL,0,4,Malphite,Lillia,Sett,Jhin,Bard,Red
4,JD,2,1,Camille,Sylas,Ekko,Jhin,Senna,Red
5,TSM,0,1,Renekton,Nidalee,Zilean,Senna,Rakan,Red
6,Suning,3,1,Jayce,Graves,Twisted Fate,Senna,Tahm Kench,Blue
7,TL,0,2,Volibear,Graves,Orianna,Kalista,Sett,Red
8,TL,1,4,Shen,Lillia,Syndra,Senna,Pan,Blue
9,Rogue,1,2,Ornn,Graves,Orianna,Kalista,Pantheon,Blue


In [5]:
le = preprocessing.LabelEncoder()

In [6]:
team = data["Team"].values
top = data["Top"].values
jungle = data["Jungle"].values
mid = data["Mid"].values
adc = data["Ad carry"].values
supp = data["Support"].values
side = data["Side"].values

In [7]:
data["Team"] = le.fit_transform(team)
data["Top"] = le.fit_transform(top)
data["Jungle"] = le.fit_transform(jungle)
data["Mid"] = le.fit_transform(mid)
data["Ad carry"] = le.fit_transform(adc)
data["Support"] = le.fit_transform(supp)
data["Side"] = le.fit_transform(side)



In [8]:
data.head(100)

,Team,Wins,Losses,Top,Jungle,Mid,Ad carry,Support,Side
0,14,0,2,15,2,6,4,1,0
1,3,3,1,18,3,9,1,5,0
2,8,1,2,4,10,13,1,7,0
3,12,0,4,8,7,11,5,1,1
4,6,2,1,0,9,2,5,11,1
...,...,...,...,...,...,...,...,...,...
73,12,0,0,10,2,9,2,6,1
74,1,0,0,11,8,11,1,9,1
75,7,2,2,13,2,16,9,7,2
76,10,0,0,8,0,7,6,10,0


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

In [10]:
rf = RandomForestClassifier(min_samples_split=2, criterion='entropy', max_depth=50, max_features='auto', n_estimators=250)

In [11]:
X = data
y = data_target
accuracy = []
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X,y)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #model
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    score = accuracy_score(pred, y_test)
    accuracy.append(score)
    
print("Accuracy: ", accuracy)

TRAIN: [ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77] TEST: [0 1 2 3 4 5 6 7]
TRAIN: [ 0  1  2  3  4  5  6  7 14 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77] TEST: [ 8  9 10 11 12 13 15 16]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 22 25 26 27 28 29 30 31
 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77] TEST: [14 17 18 19 20 21 23 24]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24
 29 30 31 34 35 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77] TEST: [22 25 26 27 28 32 33 

In [19]:
sample = pd.DataFrame({
    "Team" : [7],
    "Wins" : [0],
    "Losses" : [5],
    "Top" : [0],
    "Jungle" : [2],
    "Mid" : [15],
    "Ad carry" : [1],
    "Support" : [9],
    "Side" : [0]
})
sample_2 = pd.DataFrame({
    "Team" : [14],
    "Wins" : [0],
    "Losses" : [5],
    "Top" : [0],
    "Jungle" : [2],
    "Mid" : [15],
    "Ad carry" : [1],
    "Support" : [15],
    "Side" : [1]
})

sample_3 = pd.DataFrame({
    "Team" : [3],
    "Wins" : [3],
    "Losses" : [1],
    "Top" : [18],
    "Jungle" : [3],
    "Mid" : [9],
    "Ad carry" : [1],
    "Support" : [5],
    "Side" : [0]
})

In [20]:
pred_2 = rf.predict(sample_3)

In [21]:
pred_2[0]


1